In [2]:
import numpy as np
import matplotlib.pyplot as plt
from src.maze_functions import random_maze_generator, get_total_length, get_coord_list_plot, get_coord_list_matrix, get_no_corners, destruct_maze, repair_maze
import pandas as pd

In [2]:
maze_library = {}
repair_maze_lib = {}
zero_corner_maze_lib = {}

number_of_maze = 0
total_length_list = []

target_loc_modes = ["east", "north"]

In [3]:
step_size = 0.5
side_length = 20

N = 200

for i in range(N):
    for target_loc_mode in target_loc_modes:
        number_of_maze += 1
        maze_matrix = random_maze_generator(side_length, target_loc_mode=target_loc_mode, path_finding_mode='random')
        coordinates = get_coord_list_plot(maze_matrix)
        
        # Multiply every coordinate by step size 
        coordinates = [(coord[0] * step_size, coord[1] * step_size) for coord in coordinates]
        matrix_coordinates = get_coord_list_matrix(maze_matrix)
        
        maze_id = f"Maze_{number_of_maze}"
        coordinates_str = '*'.join([f"{coord[0]} {coord[1]}" for coord in coordinates])
        
        # Extract features
        total_length = get_total_length(matrix_coordinates, step_size)
        no_corners = get_no_corners(matrix_coordinates)

        maze_side_length = (step_size * side_length)

        # Store features
        maze_library[maze_id] = {
            "OutletPosition": target_loc_mode,
            "StepSize": step_size,
            "SideLength": maze_side_length,
            "TotalLength": total_length,
            "Corner": no_corners,
            "Coordinates": coordinates_str
        }

In [4]:
# Build the longest maze for each target location
longest_east_maze_matrix = random_maze_generator(side_length, target_loc_mode='east', path_finding_mode='longest')
longest_north_maze_matrix = random_maze_generator(side_length, target_loc_mode='north', path_finding_mode='longest')
  
longest_maze_lib = {
    "east": longest_east_maze_matrix,
    "north": longest_north_maze_matrix
}

repair_modes = ["random", "shortest"]
for target_loc_mode, maze_matrix in longest_maze_lib.items():
    
    longest_maze_matrix = longest_maze_lib[target_loc_mode]
    longest_matrix_coordinates = get_coord_list_matrix(longest_maze_matrix)
    
    largest_number_in_matrix = np.max(longest_maze_matrix)
        
    for i in range(2, largest_number_in_matrix):
        
        for mode in repair_modes:
            
            # Destruct the path from tha chosen number to the largest number and construct a new path as the shortest path
            chosen_index = largest_number_in_matrix - i
            cut_maze_matrix, locations_in_cut_maze = destruct_maze(longest_maze_matrix,
                                                                   longest_matrix_coordinates,
                                                                   chosen_index,
                                                                   largest_number_in_matrix - 1)
            
            sorted_cut_indices = np.sort([chosen_index, largest_number_in_matrix - 1])
            
                
            # Repair the maze
            repaired_maze_matrix = repair_maze(cut_maze_matrix,
                                               longest_maze_matrix,
                                               longest_matrix_coordinates,
                                               sorted_cut_indices,
                                               repair_mode=mode)
            
            number_of_maze += 1
            repaired_maze_side_length = (step_size * side_length)
            repaired_total_length = get_total_length(get_coord_list_matrix(repaired_maze_matrix), step_size)
            repaired_no_corners = get_no_corners(get_coord_list_matrix(repaired_maze_matrix))
            repaired_coordinates = get_coord_list_plot(repaired_maze_matrix)
            
            # Multiply every coordinate by step size
            repaired_coordinates = [(coord[0] * step_size, coord[1] * step_size) for coord in repaired_coordinates]
            repaired_coordinates_str = '*'.join([f"{coord[0]} {coord[1]}" for coord in repaired_coordinates])
            
            repair_maze_lib[f"Maze_{number_of_maze}"] = {
                "OutletPosition": target_loc_mode,
                "StepSize": step_size,
                "SideLength": repaired_maze_side_length,
                "TotalLength": repaired_total_length,
                "Corner": repaired_no_corners,
                "Coordinates": repaired_coordinates_str
            }

In [5]:
maze_library_df = pd.DataFrame(maze_library).T
maze_library_df.reset_index(inplace=True)
maze_library_df.rename(columns={"index": "MazeID"}, inplace=True)
maze_library_df

,MazeID,OutletPosition,StepSize,SideLength,TotalLength,Corner,Coordinates
0,Maze_1,east,0.50,10.00,66.50,84,0.0 5.0*0.5 5.0*1.0 5.0*1.0 5.5*0.5 5.5*0.5 6....
1,Maze_2,north,0.50,10.00,98.50,115,0.0 5.0*0.5 5.0*1.0 5.0*1.0 5.5*1.5 5.5*1.5 5....
2,Maze_3,east,0.50,10.00,51.50,76,0.0 5.0*0.5 5.0*0.5 5.5*1.0 5.5*1.5 5.5*1.5 6....
3,Maze_4,north,0.50,10.00,108.50,131,0.0 5.0*0.5 5.0*0.5 5.5*0.5 6.0*0.5 6.5*1.0 6....
4,Maze_5,east,0.50,10.00,91.50,108,0.0 5.0*0.5 5.0*1.0 5.0*1.0 5.5*1.0 6.0*1.0 6....
...,...,...,...,...,...,...,...
395,Maze_396,north,0.50,10.00,69.50,93,0.0 5.0*0.5 5.0*0.5 5.5*0.5 6.0*0.5 6.5*0.5 7....
396,Maze_397,east,0.50,10.00,50.50,64,0.0 5.0*0.5 5.0*1.0 5.0*1.5 5.0*1.5 5.5*1.0 5....
397,Maze_398,north,0.50,10.00,60.50,73,0.0 5.0*0.5 5.0*1.0 5.0*1.0 5.5*1.5 5.5*1.5 6....
398,Maze_399,east,0.50,10.00,61.50,74,0.0 5.0*0.5 5.0*1.0 5.0*1.0 5.5*1.0 6.0*0.5 6....


In [6]:
repaired_maze_library_df = pd.DataFrame(repair_maze_lib).T
repaired_maze_library_df.reset_index(inplace=True)
repaired_maze_library_df.rename(columns={"index": "MazeID"}, inplace=True)
repaired_maze_library_df

,MazeID,OutletPosition,StepSize,SideLength,TotalLength,Corner,Coordinates
0,Maze_401,east,0.50,10.00,180.50,86,0.0 5.0*0.5 5.0*0.5 5.5*0.5 6.0*0.5 6.5*0.5 7....
1,Maze_402,east,0.50,10.00,180.50,86,0.0 5.0*0.5 5.0*0.5 5.5*0.5 6.0*0.5 6.5*0.5 7....
2,Maze_403,east,0.50,10.00,180.50,86,0.0 5.0*0.5 5.0*0.5 5.5*0.5 6.0*0.5 6.5*0.5 7....
3,Maze_404,east,0.50,10.00,180.50,86,0.0 5.0*0.5 5.0*0.5 5.5*0.5 6.0*0.5 6.5*0.5 7....
4,Maze_405,east,0.50,10.00,180.50,86,0.0 5.0*0.5 5.0*0.5 5.5*0.5 6.0*0.5 6.5*0.5 7....
...,...,...,...,...,...,...,...
1431,Maze_1832,north,0.50,10.00,11.50,17,0.0 5.0*0.5 5.0*0.5 4.5*1.0 4.5*1.0 5.0*1.0 5....
1432,Maze_1833,north,0.50,10.00,65.50,81,0.0 5.0*0.5 5.0*0.5 4.5*1.0 4.5*1.0 4.0*1.0 3....
1433,Maze_1834,north,0.50,10.00,10.50,17,0.0 5.0*0.5 5.0*0.5 5.5*0.5 6.0*1.0 6.0*1.0 6....
1434,Maze_1835,north,0.50,10.00,56.50,67,0.0 5.0*0.5 5.0*0.5 5.5*0.5 6.0*0.5 6.5*0.5 7....


In [7]:
# merge the two libraries
merged_maze_library_df = pd.concat([maze_library_df, repaired_maze_library_df], ignore_index=True)
merged_maze_library_df

,MazeID,OutletPosition,StepSize,SideLength,TotalLength,Corner,Coordinates
0,Maze_1,east,0.50,10.00,66.50,84,0.0 5.0*0.5 5.0*1.0 5.0*1.0 5.5*0.5 5.5*0.5 6....
1,Maze_2,north,0.50,10.00,98.50,115,0.0 5.0*0.5 5.0*1.0 5.0*1.0 5.5*1.5 5.5*1.5 5....
2,Maze_3,east,0.50,10.00,51.50,76,0.0 5.0*0.5 5.0*0.5 5.5*1.0 5.5*1.5 5.5*1.5 6....
3,Maze_4,north,0.50,10.00,108.50,131,0.0 5.0*0.5 5.0*0.5 5.5*0.5 6.0*0.5 6.5*1.0 6....
4,Maze_5,east,0.50,10.00,91.50,108,0.0 5.0*0.5 5.0*1.0 5.0*1.0 5.5*1.0 6.0*1.0 6....
...,...,...,...,...,...,...,...
1831,Maze_1832,north,0.50,10.00,11.50,17,0.0 5.0*0.5 5.0*0.5 4.5*1.0 4.5*1.0 5.0*1.0 5....
1832,Maze_1833,north,0.50,10.00,65.50,81,0.0 5.0*0.5 5.0*0.5 4.5*1.0 4.5*1.0 4.0*1.0 3....
1833,Maze_1834,north,0.50,10.00,10.50,17,0.0 5.0*0.5 5.0*0.5 5.5*0.5 6.0*1.0 6.0*1.0 6....
1834,Maze_1835,north,0.50,10.00,56.50,67,0.0 5.0*0.5 5.0*0.5 5.5*0.5 6.0*0.5 6.5*0.5 7....


In [8]:
# drop duplicates by StepSize, SideLength, TotalLength, Corner
merged_maze_library_no_duplicates = merged_maze_library_df.drop_duplicates(subset=["OutletPosition", "StepSize", "SideLength", "TotalLength", "Corner"])
merged_maze_library_no_duplicates.reset_index(drop=True, inplace=True)
merged_maze_library_no_duplicates

,MazeID,OutletPosition,StepSize,SideLength,TotalLength,Corner,Coordinates
0,Maze_1,east,0.50,10.00,66.50,84,0.0 5.0*0.5 5.0*1.0 5.0*1.0 5.5*0.5 5.5*0.5 6....
1,Maze_2,north,0.50,10.00,98.50,115,0.0 5.0*0.5 5.0*1.0 5.0*1.0 5.5*1.5 5.5*1.5 5....
2,Maze_3,east,0.50,10.00,51.50,76,0.0 5.0*0.5 5.0*0.5 5.5*1.0 5.5*1.5 5.5*1.5 6....
3,Maze_4,north,0.50,10.00,108.50,131,0.0 5.0*0.5 5.0*0.5 5.5*0.5 6.0*0.5 6.5*1.0 6....
4,Maze_5,east,0.50,10.00,91.50,108,0.0 5.0*0.5 5.0*1.0 5.0*1.0 5.5*1.0 6.0*1.0 6....
...,...,...,...,...,...,...,...
1534,Maze_1830,north,0.50,10.00,12.50,17,0.0 5.0*0.5 5.0*0.5 4.5*0.5 4.0*1.0 4.0*1.0 4....
1535,Maze_1831,north,0.50,10.00,24.50,23,0.0 5.0*0.5 5.0*0.5 4.5*1.0 4.5*1.0 5.0*1.5 5....
1536,Maze_1832,north,0.50,10.00,11.50,17,0.0 5.0*0.5 5.0*0.5 4.5*1.0 4.5*1.0 5.0*1.0 5....
1537,Maze_1834,north,0.50,10.00,10.50,17,0.0 5.0*0.5 5.0*0.5 5.5*0.5 6.0*1.0 6.0*1.0 6....


In [9]:
for i in range(len(merged_maze_library_no_duplicates)):
    
    maze_id = merged_maze_library_no_duplicates.loc[i, "MazeID"]
    outlet_position = "east"
    step_size = merged_maze_library_no_duplicates.loc[i, "StepSize"]
    side_length = merged_maze_library_no_duplicates.loc[i, "SideLength"]
    total_length = merged_maze_library_no_duplicates.loc[i, "TotalLength"]
    corner = merged_maze_library_no_duplicates.loc[i, "Corner"]
    coordinates_str = merged_maze_library_no_duplicates.loc[i, "Coordinates"]
    
    
    if total_length not in total_length_list and corner != 0:
        total_length_list.append(total_length)
        
        start_point = (0.0, 5.0)
        target_point = (total_length, 5.0)
        
        # Create A list from start to target point with step size 0.5
        zero_corner_coordinates = [(x, 5.0) for x in np.arange(0.0, total_length + 0.5, 0.5)]
        zero_corner_coordinates_str = '*'.join([f"{coord[0]} {coord[1]}" for coord in zero_corner_coordinates])
        
        number_of_maze += 1
        zero_corner_maze_lib[f"Maze_{number_of_maze}"] = {
            "OutletPosition": outlet_position,
            "StepSize": step_size,
            "SideLength": total_length,
            "TotalLength": total_length,
            "Corner": 0,
            "Coordinates": zero_corner_coordinates_str
        }

In [10]:
zero_corner_maze_library_df = pd.DataFrame(zero_corner_maze_lib).T
zero_corner_maze_library_df.reset_index(inplace=True)
zero_corner_maze_library_df.rename(columns={"index": "MazeID"}, inplace=True)
zero_corner_maze_library_df

,MazeID,OutletPosition,StepSize,SideLength,TotalLength,Corner,Coordinates
0,Maze_1837,east,0.50,66.50,66.50,0,0.0 5.0*0.5 5.0*1.0 5.0*1.5 5.0*2.0 5.0*2.5 5....
1,Maze_1838,east,0.50,98.50,98.50,0,0.0 5.0*0.5 5.0*1.0 5.0*1.5 5.0*2.0 5.0*2.5 5....
2,Maze_1839,east,0.50,51.50,51.50,0,0.0 5.0*0.5 5.0*1.0 5.0*1.5 5.0*2.0 5.0*2.5 5....
3,Maze_1840,east,0.50,108.50,108.50,0,0.0 5.0*0.5 5.0*1.0 5.0*1.5 5.0*2.0 5.0*2.5 5....
4,Maze_1841,east,0.50,91.50,91.50,0,0.0 5.0*0.5 5.0*1.0 5.0*1.5 5.0*2.0 5.0*2.5 5....
...,...,...,...,...,...,...,...
166,Maze_2003,east,0.50,13.50,13.50,0,0.0 5.0*0.5 5.0*1.0 5.0*1.5 5.0*2.0 5.0*2.5 5....
167,Maze_2004,east,0.50,12.50,12.50,0,0.0 5.0*0.5 5.0*1.0 5.0*1.5 5.0*2.0 5.0*2.5 5....
168,Maze_2005,east,0.50,11.50,11.50,0,0.0 5.0*0.5 5.0*1.0 5.0*1.5 5.0*2.0 5.0*2.5 5....
169,Maze_2006,east,0.50,178.50,178.50,0,0.0 5.0*0.5 5.0*1.0 5.0*1.5 5.0*2.0 5.0*2.5 5....


In [11]:
# merge the two libraries
final_merged_maze_library_df = pd.concat([merged_maze_library_no_duplicates, zero_corner_maze_library_df], ignore_index=True)
final_merged_maze_library_df

,MazeID,OutletPosition,StepSize,SideLength,TotalLength,Corner,Coordinates
0,Maze_1,east,0.50,10.00,66.50,84,0.0 5.0*0.5 5.0*1.0 5.0*1.0 5.5*0.5 5.5*0.5 6....
1,Maze_2,north,0.50,10.00,98.50,115,0.0 5.0*0.5 5.0*1.0 5.0*1.0 5.5*1.5 5.5*1.5 5....
2,Maze_3,east,0.50,10.00,51.50,76,0.0 5.0*0.5 5.0*0.5 5.5*1.0 5.5*1.5 5.5*1.5 6....
3,Maze_4,north,0.50,10.00,108.50,131,0.0 5.0*0.5 5.0*0.5 5.5*0.5 6.0*0.5 6.5*1.0 6....
4,Maze_5,east,0.50,10.00,91.50,108,0.0 5.0*0.5 5.0*1.0 5.0*1.0 5.5*1.0 6.0*1.0 6....
...,...,...,...,...,...,...,...
1705,Maze_2003,east,0.50,13.50,13.50,0,0.0 5.0*0.5 5.0*1.0 5.0*1.5 5.0*2.0 5.0*2.5 5....
1706,Maze_2004,east,0.50,12.50,12.50,0,0.0 5.0*0.5 5.0*1.0 5.0*1.5 5.0*2.0 5.0*2.5 5....
1707,Maze_2005,east,0.50,11.50,11.50,0,0.0 5.0*0.5 5.0*1.0 5.0*1.5 5.0*2.0 5.0*2.5 5....
1708,Maze_2006,east,0.50,178.50,178.50,0,0.0 5.0*0.5 5.0*1.0 5.0*1.5 5.0*2.0 5.0*2.5 5....


In [12]:
# shuffle the rows
final_merged_maze_library_df = final_merged_maze_library_df.sample(frac=1).reset_index(drop=True)
final_merged_maze_library_df

,MazeID,OutletPosition,StepSize,SideLength,TotalLength,Corner,Coordinates
0,Maze_1467,north,0.50,10.00,125.50,61,0.0 5.0*0.5 5.0*0.5 4.5*0.5 4.0*0.5 3.5*0.5 3....
1,Maze_1316,north,0.50,10.00,136.50,45,0.0 5.0*0.5 5.0*0.5 4.5*0.5 4.0*0.5 3.5*0.5 3....
2,Maze_990,east,0.50,10.00,44.50,28,0.0 5.0*0.5 5.0*0.5 5.5*0.5 6.0*0.5 6.5*0.5 7....
3,Maze_152,north,0.50,10.00,45.50,59,0.0 5.0*0.5 5.0*0.5 5.5*0.5 6.0*1.0 6.0*1.0 6....
4,Maze_733,east,0.50,10.00,128.50,82,0.0 5.0*0.5 5.0*0.5 5.5*0.5 6.0*0.5 6.5*0.5 7....
...,...,...,...,...,...,...,...
1705,Maze_694,east,0.50,10.00,113.50,56,0.0 5.0*0.5 5.0*0.5 5.5*0.5 6.0*0.5 6.5*0.5 7....
1706,Maze_68,north,0.50,10.00,68.50,91,0.0 5.0*0.5 5.0*0.5 4.5*0.5 4.0*0.5 3.5*1.0 3....
1707,Maze_1993,east,0.50,128.50,128.50,0,0.0 5.0*0.5 5.0*1.0 5.0*1.5 5.0*2.0 5.0*2.5 5....
1708,Maze_33,east,0.50,10.00,100.50,122,0.0 5.0*0.5 5.0*0.5 5.5*0.5 6.0*0.5 6.5*0.5 7....


In [3]:
# plot TotalLength graph histogram
plt.hist(final_merged_maze_library_df["TotalLength"], bins=50, color='blue', edgecolor='black')
plt.xlabel('Total Length')
plt.ylabel('Number of Mazes')
plt.title('Total Length Histogram')
plt.show()

NameError: name 'final_merged_maze_library_df' is not defined

In [14]:
TOTAL_MAZE_NUM = len(final_merged_maze_library_df)
TOTAL_DATA = TOTAL_MAZE_NUM * 192
EACH_SIM_TIME = 8
TOTAL_SIM_DAY = TOTAL_DATA * EACH_SIM_TIME / (60 * 60 * 24)

print(f"Total Maze Number: {TOTAL_MAZE_NUM}")
print(f"Total Data: {TOTAL_DATA}")
print(f"Total Simulation Time: {TOTAL_SIM_DAY} days")

Total Maze Number: 1710
Total Data: 328320
Total Simulation Time: 30.4 days


In [15]:
# save the final merged maze library
final_merged_maze_library_df.to_csv("new_maze_library.csv", index=False)